In [10]:
import subprocess, json, pandas as pd

In [7]:
def install_and_import_libraries(list_of_libraries):
    for library in list_of_libraries:
        try:
            # Installer la librairie
            result = subprocess.run(["pip", "install", library, "--user"], capture_output=True, text=True, check=True)
            for line in result.stdout.splitlines():
                if not line.startswith("Requirement already satisfied"):
                    print(line)
        except Exception as e:
            print(f"Error installing {library}: {e}")

        try:
            # Importer la librairie après installation
            globals()[library] = __import__(library)
        except Exception as e:
            print(f"Error importing {library}: {e}")

In [ ]:

"""
import os
try:
    notebook_path = os.path.join(os.getcwd(), r'C:\Users\f.gionnane\Downloads\Projects\Oceanography-Project-WIP\getMarineFromApi.ipynb')

    subprocess.run(["jupyter", "nbconvert", "--execute", "--to", "notebook", "--inplace", notebook_path], check=True)
except subprocess.CalledProcessError as e:
    print(e)


    """

In [8]:
libraries =["google-api-python-client", "google-auth", "google-auth-oauthlib", "google-auth-httplib2", "gspread"]
install_and_import_libraries(libraries)

from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import gspread

scopes = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/drive.file',
    'https://www.googleapis.com/auth/drive.appdata','https://www.googleapis.com/auth/drive.appfolder'
]

# Chemin vers le fichier de crédentials JSON
path_to_google_cred = r"C:\Users\f.gionnane\Downloads\Projects\google_sheets_api.json"

# Créer les crédentials à partir du fichier de service account
creds = Credentials.from_service_account_file(path_to_google_cred, scopes=scopes)

# Autoriser l'accès en utilisant les crédentials
client = gspread.authorize(creds)

# ID de la feuille Google Sheets
sheet_id = "1rXyD4WW5DcrO2iCywfwLbqps3Z-RumxQH9hJofhVM24"

# Ouvrir la feuille Google Sheets en utilisant son ID
spreadsheet = client.open_by_key(sheet_id)
sheet_data_marine = spreadsheet.get_worksheet(0)
sheet_data_meteo = spreadsheet.get_worksheet(1)

Error importing google-api-python-client: No module named 'google-api-python-client'
Error importing google-auth: No module named 'google-auth'
Error importing google-auth-oauthlib: No module named 'google-auth-oauthlib'
Error importing google-auth-httplib2: No module named 'google-auth-httplib2'


In [21]:
import pandas as pd

# Charger les données des feuilles
df_marine = pd.DataFrame(sheet_data_marine.get_all_records())
df_meteo = pd.DataFrame(sheet_data_meteo.get_all_records())

def clean_and_merge(df1, df2):
    # Spécifie le format de date si connu
    date_format = '%d-%m-%Y'  # Change-le si nécessaire
    
    # Normalisation des dates
    df1['Date'] = pd.to_datetime(df1['Date'], format=date_format, errors='coerce')
    df2['Date'] = pd.to_datetime(df2['Date'], format=date_format, errors='coerce')

    # Nettoyage des heures
    df1['Heure'] = df1['Heure'].astype(str).str.strip().str.zfill(2) + ':00'
    df2['Heure'] = df2['Heure'].astype(str).str.strip().str.zfill(2) + ':00'

    # Suppression des valeurs invalides
    df1 = df1.dropna(subset=['Date', 'Heure'])
    df2 = df2.dropna(subset=['Date', 'Heure'])

    # Fusion sur Date et Heure
    merged_df = pd.merge(df1, df2, on=['Date', 'Heure'], how='inner')
    
    return merged_df


df = clean_and_merge(df_marine,df_meteo)
df

,Wind Dir (°),Wind Spd (ms),Wind Gusts 1h (ms),Wave Height (m),Dominant Wave Period (s),Avg Wave Period (s),Dominant Wave Dir,Pressure,Air Temp,Water Temp,...,Date,Heure,wind dir (°),,wind gusts 1h (ms),T° (C°),T° max 24h (C°),T° min 24h (°),sea level pressure (hPa),Precip 1h (mm)


In [16]:
df1.head()

,Wind Dir (°),Wind Spd (ms),Wind Gusts 1h (ms),Wave Height (m),Dominant Wave Period (s),Avg Wave Period (s),Dominant Wave Dir,Pressure,Air Temp,Water Temp,dewpoint,Date,Heure
0,20,6,9,2.9,13,8.3,311,1020.8,21.5,24.1,03/02/2025,22:00,
1,30,6,8,2.9,15,8.6,40,1020.7,21.4,24.1,03/02/2025,21:50,
2,20,7,9,2.9,13,8.3,311,1020.6,21.5,24.1,03/02/2025,21:40,
3,20,7,10,2.9,13,8.3,311,1020.7,21.4,24.1,03/02/2025,21:30,
4,20,7,10,3.1,13,8.6,31,1020.6,21.4,24.1,03/02/2025,21:20,
